# **Prepocessing Teks**

Normalisasi text
1. Punctuation process
2. Tokenisasi
3. Stopword
4. Steeming
5. Feature extraction dan membentuk VSM dalam term frequency

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

prod = '10'
page = 1

# membuat file CSV untuk menulis hasil scraping
with open('hasil_scraping.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Judul', 'Penulis', 'Dosen Pembimbing I', 'Dosen Pembimbing II', 'Abstrak']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    while True:
        base_url = 'https://pta.trunojoyo.ac.id/c_search/byprod/{}/{}'.format(prod, page)
        url = base_url
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')
        items = soup.find_all('li', attrs={'data-id': 'id-1'})

        if not items:
            # Jika tidak ada item di halaman saat ini, keluar dari loop
            break

        for it in items:
            data = {}
            title = it.find('a', class_='title').text
            data['Judul'] = title
            div_elements = it.find_all('div', style='padding:2px 2px 2px 2px;')
            for div in div_elements:
                span = div.find('span')
                if span:
                    span_text = span.get_text()
                    key, value = span_text.split(':', 1)
                    data[key.strip()] = value.strip()

            # Mengambil link abstrak dari elemen dengan kelas 'gray button'
            abstrak_button = it.find('a', class_='gray button')
            if abstrak_button:
                abstrak_link = abstrak_button['href']
                abstrak_req = requests.get(abstrak_link)
                abstrak_soup = BeautifulSoup(abstrak_req.text, 'html.parser')
                abstrak = abstrak_soup.find('p', align='justify')
                if abstrak:
                    abstrak_text = abstrak.get_text(strip=True)
                    data['Abstrak'] = abstrak_text
                else:
                    data['Abstrak'] = "Tidak ditemukan abstrak"

            # Menulis data ke dalam file CSV
            writer.writerow(data)
            print("Data berhasil ditambahkan:", data)

        page += 1

print("Scraping selesai dan hasil disimpan dalam hasil_scraping.csv")


Data berhasil ditambahkan: {'Judul': 'PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \r\nTERDISTRIBUSI MENGGUNAKAN ORACLE STUDI KASUS \r\nSIAKAD UNIVERSITAS TRUNOJOYO', 'Penulis': 'A.Ubaidillah S.Kom', 'Dosen Pembimbing I': 'Budi Setyono M.T', 'Dosen Pembimbing II': 'Hermawan S.T', 'Abstrak': 'Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik. Di Universitas Trunojoyo telah tersedia SIAKAD,  namun  masih  menggunakan  database  terpusat. Sistem seperti ini memberikan kelebihan yaitu perawatannya mudah  selain  itu  juga  membutuhkan  sedikit  biaya,  namun sistem  tersebut  juga  berpotensi  mengahadapi  kendala-kendala  yaitu  dalam  proses  transaksi  data  karena  padatnya jaringan yang menuju database SIAKAD, kelambatan dalam pemrosesan  respon  query 

In [ ]:
import pandas as pd
# abst = pd.read_csv('/content/hasil_scraping.csv')
abst = pd.read_csv("https://raw.githubusercontent.com/alisaSugiarti/ppw/main/hasil_scraping.csv")
abst.head()

,Judul,Penulis,Dosen Pembimbing I,Dosen Pembimbing II,Abstrak
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...


In [ ]:
import requests
import nltk
import string
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pandas as pd
import re

nltk.download('stopwords')
nltk.download('punkt')


def download_custom_stopwords(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        stopwords_text = response.text
        custom_stopwords = set(stopwords_text.splitlines())
        return custom_stopwords
    except requests.exceptions.RequestException as e:
        print("Gagal mengunduh daftar kata-kata stop words:", e)
        return set()


github_stopwords_url = 'https://raw.githubusercontent.com/alisaSugiarti/ppw/main/daftar_stopword.txt'

# Mengunduh daftar kata-kata stop words dari GitHub
custom_stopwords = download_custom_stopwords(github_stopwords_url)

# Import stopwords dalam bahasa Indonesia
from nltk.corpus import stopwords
stop_words = set(stopwords.words('indonesian'))

# Gabungkan stopwords bawaan dengan custom_stopwords
stop_words.update(custom_stopwords)

# fungsi untuk menghapus tanda baca dan stopwords
def preprocess_text(teks):
    if pd.isna(teks):
        teks = ""
    else:
        # Menghapus angka
        num_clean = re.sub(r"\d+", "", teks)

        # Punctuation process (Hapus tanda baca)
        teks_clean = num_clean.translate(str.maketrans('', '', '%', string.punctuation))

        # Tokenisasi
        tokens = nltk.word_tokenize(teks_clean)

        # Stopwords removal
        teks_cleaned = [word for word in tokens if word.lower() not in stop_words]

        # Stemming
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(word) for word in teks_cleaned]

        # Gabung kembali tokens menjadi teks
        teks = ' '.join(stemmed_tokens)

    return teks

# Menambahkan kolom untuk setiap tahap preprocessing
abst['cleaning'] = abst['Abstrak'].apply(preprocess_text)  # Casefolding, Tokenizing, Stopword removal, dan Stemming
abst['tokenized'] = abst['cleaning'].apply(nltk.word_tokenize)  # Tokenizing
abst['stopwords_removed'] = abst['tokenized'].apply(lambda x: [word for word in x if word.lower() not in stop_words])  # Stopword removal
abst['stemmed'] = abst['stopwords_removed'].apply(lambda x: [PorterStemmer().stem(word) for word in x])

# Simpan hasil preprocessing ke dalam file XLSX
abst.to_excel('HasilPreposPTA.xlsx', index=False)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Ekstraksi fitur dan membentuk VSM dalam term frequency
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(abst['cleaning'])
# print(count_matrix)
count_array = count_matrix.toarray()
df = pd.DataFrame(data=count_array,columns = vectorizer.vocabulary_.keys())
df


,sistem,informasi,akademik,siakad,berfungsi,menangani,pengelolaan,penyajian,datadata,fakulta,...,from,acceler,augmentasi,weak,stump,ransel,diseimbangkan,detector,anchor,pretrain
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import numpy as np

kata = set(vectorizer.vocabulary_.keys())
vocab_to_int = {word: idx for idx, word in enumerate(kata)}
int_to_vocab = {idx: word for idx, word in enumerate(kata)}

# Menghitung frekuensi kata
word_counts = Counter(kata)
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)

vocab_size = len(kata)
embedding_dim = 100
# print(kata)

# Menginisialisasi model Skip-gram
class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, target):
        target_embeds = self.embeddings(target)
        output = self.linear(target_embeds)
        return output

# Membuat instance model Skip-gram
model = SkipGram(vocab_size, embedding_dim)

# Contoh penggunaan kata-kata sebagai keys dari dictionary
print("Kata-kata dalam vectorizer:")
print(kata)

# Untuk melihat struktur model Anda, Anda dapat mencetaknya dengan perintah berikut:
print("\nStruktur Model Skip-gram:")
print(model)

# Fungsi untuk training model
def train(model, data, vocab_to_int, epochs=5, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        total_loss = 0

        for sentence in data:
            # Memisahkan kata-kata dalam kalimat
            words = sentence.split()

            for target_word in words:
                # Memeriksa apakah kata ada dalam vocabulary
                if target_word in vocab_to_int:
                    target = torch.LongTensor([vocab_to_int[target_word]])
                    optimizer.zero_grad()
                    output = model(target)
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss}')

# Training model
data = abst['abstrak_cleaned']
train(model, data, vocab_to_int)


# Mendapatkan vektor kata
word_vector = model.embeddings.weight
word_vector = word_vector.detach().numpy()

# Contoh penggunaan vektor kata
word_to_find = "skip-gram"
word_idx = vocab_to_int[word_to_find]
word_vector[word_idx]

Kata-kata dalam vectorizer:
{'internet', 'antraquinon', 'meluasnya', 'responsiven', 'psikomotorik', 'merupaka', 'alfabet', 'want', 'arena', 'xl', '17676', 'mengaji', 'fotografi', 'berkepanjangan', '24774', 'asupan', 'kbliklasifikasi', 'makmur', 'otp', 'berobat', 'biomekanika', 'iowa', 'anggotanya', '5w1h', 'regenerasi', 'elemen', 'from', 'hole', 'dbi', 'petualangan', 'insinyur', '2001', 'darma', 'bri', 'penyediaanpendampingandanpendanaanuntukmendoronginovasimasyarakatdalamupayapenanggulangankemiskinan', 'bidak', 'sembarangan', 'kebanyakan', 'seolaholah', 'perangkatperangkat', 'random', 'maduraâ', 'langsung', 'canberra', 'utnuk', 'lahir', 'bangkalanpenggunaan', 'file', 'masingmas', 'diband', 'similarityuntuk', 'kunciâ', 'oknum', 'railindo', 'panasan', '232', 'ditengahtengah', 'seimbangan', 'logic', 'mengkombinasikan', 'posyandu', 'mempelajari', 'mapu', 'fourier', 'menelaah', 'shoot', 'sumberdaya', 'risk', 'kitab', 'berhasil', 'tracker', 'androiduniversita', 'capai', 'bacaan', 'tergana',

In [ ]:
import numpy as np

# menghitung jumlah kemunculan kata untuk setiap kata dalam VSM dalam term frequency
word_counts = np.sum(count_matrix.toarray(), axis=0)

# membuat pasangan kata dan jumlah kemunculannya dalam bentuk kamus (dictionary)
word_count_dict = dict(zip(vectorizer.get_feature_names_out(), word_counts))

# Tampilkan kamus kata dan jumlah kemunculannya
print("\nJumlah Kemunculan Kata:")
for word, count in word_count_dict.items():
    print(f"{word}: {count}")

Output streaming akan dipotong hingga 5000 baris terakhir.
mengawasi: 2
mengawetkan: 1
mengawinkan: 1
mengecek: 4
mengecilnya: 1
mengedepankan: 2
mengedit: 1
mengedukasi: 1
mengefektikan: 1
mengefisienkan: 1
mengefisiensi: 1
mengejek: 1
mengekonomiskan: 1
mengekplorasi: 1
mengeksplisitkan: 1
mengekstraksi: 3
mengeliminasi: 2
mengelola: 26
mengelolah: 1
mengelolanya: 1
mengelompokan: 6
mengelompokkan: 14
mengelompokkannya: 1
mengeluarkan: 9
mengeluh: 1
mengeluhkan: 1
mengema: 1
mengembalikan: 1
mengembangakan: 1
mengembangan: 1
mengembangkan: 38
mengembangkannya: 1
mengemudi: 1
mengen: 20
mengenali: 16
mengenalinya: 1
mengenalkan: 3
mengendalikan: 13
mengenkripsi: 3
mengentaskan: 2
mengenyam: 1
mengerahkan: 1
mengerjakannya: 1
mengerti: 13
mengestraksi: 1
mengetahuinya: 1
mengetik: 2
mengetikkan: 1
mengevaluasi: 16
menggabungkan: 11
menggali: 4
menggambar: 2
menggambarkan: 10
menggambarkannya: 1
mengganggu: 2
menggangu: 1
mengganti: 3
menggantiganti: 1
menggantikan: 6
menggelind: 5
meng

In [ ]:
def nltk_frequency_vectorize(corpus):

    # The NLTK frequency vectorize method
    from collections import defaultdict

    def vectorize(doc):
        features = defaultdict(int)

        for token in tokenize(doc):
            features[token] += 1

        return features

    return map(vectorize, corpus)
vectnltk=nltk_frequency_vectorize(abst['abstrak_cleaned'])
type(vectnltk)

map

In [ ]:
def sklearn_frequency_vectorize(corpus):
    # The Scikit-Learn frequency vectorize method
    from sklearn.feature_extraction.text import CountVectorizer

    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(corpus)
vectsklen=sklearn_frequency_vectorize(abst['abstrak_cleaned'])
print(vectsklen)

  (0, 10615)	7
  (0, 4867)	3
  (0, 1424)	3
  (0, 10512)	4
  (0, 2092)	1
  (0, 6962)	1
  (0, 8713)	1
  (0, 8912)	1
  (0, 2948)	1
  (0, 4089)	2
  (0, 3123)	1
  (0, 8317)	1
  (0, 6484)	1
  (0, 6834)	2
  (0, 11764)	1
  (0, 11608)	1
  (0, 11362)	1
  (0, 2944)	4
  (0, 11349)	2
  (0, 5489)	1
  (0, 9009)	1
  (0, 7692)	1
  (0, 2333)	1
  (0, 2237)	1
  (0, 7074)	1
  :	:
  (999, 1020)	1
  (999, 9375)	4
  (999, 6220)	2
  (999, 9038)	1
  (999, 11582)	2
  (999, 1152)	1
  (999, 5516)	1
  (999, 3693)	1
  (999, 6419)	1
  (999, 5752)	1
  (999, 4640)	3
  (999, 5305)	1
  (999, 4358)	1
  (999, 3917)	1
  (999, 9463)	1
  (999, 4341)	1
  (999, 6014)	1
  (999, 1619)	3
  (999, 79)	1
  (999, 6)	1
  (999, 83)	1
  (999, 5329)	1
  (999, 205)	1
  (999, 7)	1
  (999, 190)	1


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#coun_vect = CountVectorizer()
coun_vect = CountVectorizer()
count_matrix = coun_vect.fit_transform(abst['abstrak_cleaned'])
count_array = count_matrix.toarray()
df = pd.DataFrame(data=count_array,columns = coun_vect.vocabulary_.keys())
df


,sistem,informasi,akademik,siakad,berfungsi,menangani,pengelolaan,penyajian,datadata,fakulta,...,gaug,kran,ann,0167,00002,018952,ke7,094969,000031,08375
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
